# Day 17: Two Steps Forward

## Part One

You're trying to access a secure vault protected by a `4x4` grid of small rooms connected by doors. You start in the top-left room (marked `S`), and you can access the vault (marked `V`) once you reach the bottom-right room:

```
#########
#S| | | #
#-#-#-#-#
# | | | #
#-#-#-#-#
# | | | #
#-#-#-#-#
# | | |  
####### V
```

Fixed walls are marked with `#`, and doors are marked with `-` or `|`.

The doors in your current room are either open or closed (and locked) based on the hexadecimal MD5 hash of a passcode (your puzzle input) followed by a sequence of uppercase characters representing the path you have taken so far (`U` for up, `D` for down, `L` for left, and `R` for right).

Only the first four characters of the hash are used; they represent, respectively, the doors up, down, left, and right from your current position. Any `b`, `c`, `d`, `e`, or `f` means that the corresponding door is open; any other character (any number or `a`) means that the corresponding door is closed and locked.

To access the vault, all you need to do is reach the bottom-right room; reaching this room opens the vault and all doors in the maze.

For example, suppose the passcode is `hijkl`. Initially, you have taken no steps, and so your path is empty: you simply find the MD5 hash of `hijkl` alone. The first four characters of this hash are `ced9`, which indicate that up is open (`c`), down is open (`e`), left is open (`d`), and right is closed and locked (`9`). Because you start in the top-left corner, there are no "up" or "left" doors to be open, so your only choice is down.

Next, having gone only one step (down, or `D`), you find the hash of `hijklD`. This produces `f2bc`, which indicates that you can go back up, left (but that's a wall), or right. Going right means hashing `hijklDR` to get `5745` - all doors closed and locked. However, going up instead is worthwhile: even though it returns you to the room you started in, your path would then be `DU`, opening a different set of doors.

After going `DU` (and then hashing `hijklDU` to get `528e`), only the right door is open; after going `DUR`, all doors lock. (Fortunately, your actual passcode is not `hijkl`).

Passcodes actually used by Easter Bunny Vault Security do allow access to the vault if you know the right path. For example:

* If your passcode were `ihgpwlah`, the shortest path would be `DDRRRD`.

* With `kglvqrro`, the shortest path would be `DDUDRLRRUDRD`.

* With `ulqzkmiv`, the shortest would be `DRURDRUDDLLDLUURRDULRLDUUDDDRR`.

Given your vault's passcode, what is the shortest path (the actual path, not just the length) to reach the vault?

In [1]:
from hashlib import md5

inputs = 'pxxbnzuo'
path = ''
open_door = {'b', 'c', 'd', 'e', 'f'}
directions = ['U', 'D', 'L', 'R']

encoding = 'utf-8'
md5_h = lambda x, i: md5((x+str(i)).encode(encoding)).hexdigest()[:4]
valid = lambda x, i: [d for j, d in enumerate(directions) if (md5_h(x,i)[j] in open_door)]
in_bounds = lambda p: (0 <= p.count('D') - p.count('U') < 4) and (0 <= p.count('R') - p.count('L') < 4)
complete = lambda p: (p.count('D') - p.count('U') == 3) and (p.count('R') - p.count('L') == 3)

In [2]:
def explore(passcode, paths=None):
    """
    Given a passcode this function recursiveley explores the grid until it reaches the vault
    and returns the first path that is valid using a breadth first search algorithm.
    """
    # Initialise
    options = set()
    if paths is None:
        paths = {''}
    
    # Iterate over paths
    for path in paths:
        for option in valid(passcode, path):
            p = path + option
            if complete(p):
                # Return once first solution is found
                return p
            elif in_bounds(p):
                options.add(p)
    
    # Recurse if no solution
    return explore(passcode, options)

# Solution
print(f"The shortest path to the vault is: {explore(inputs)}.")

The shortest path to the vault is: RDULRDDRRD.


---

### Part Two

You're curious how robust this security solution really is, and so you decide to find longer and longer paths which still provide access to the vault. You remember that paths always end the first time they reach the bottom-right room (that is, they can never pass through it, only end in it).

For example:

* If your passcode were `ihgpwlah`, the longest path would take `370` steps.

* With `kglvqrro`, the longest path would be `492` steps long.

* With `ulqzkmiv`, the longest path would be `830` steps long.

* What is the length of the longest path that reaches the vault?

---

In [3]:
def explore_all(passcode, paths=None, lengths=None):
    """
    Given a passcode this function recursiveley explores the grid and returns 
    the length of the last valid path that reaches the goal.
    """
    # Initialise
    options = set()
    if paths is None:
        paths = {''}
    if lengths is None:
        lengths = {}
    
    # Iterate over paths
    for path in paths:
        for option in valid(passcode, path):
            p = path + option
            if complete(p):
                # Add path to dictionary if it finds vault
                lengths[p] = len(p)
            elif in_bounds(p):
                options.add(p)
    
    # Continue searching if there are still valid paths, else end search
    if options:
        return explore_all(passcode, options, lengths)
    else:
        return max(lengths.values())

# Solution
print(f"The longest valid path to the vault is length {explore_all(inputs):,}.")

The longest valid path to the vault is length 752.


---